In [1]:
#with delta features
import pickle
# Load it back
with open("speech_commands_mfcc_delta_features_torchaudio.pkl", "rb") as f:
    mfcc_features, labels, label_map = pickle.load(f)

In [2]:
import torch
from collections import OrderedDict
from torch.utils.data import random_split

# Create a subset of 5000 signals
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

subset_size = 38546
indices = torch.randperm(len(labels))[:subset_size]

# Sort the indices
sorted_indices = torch.sort(indices).values

labels = torch.tensor(labels, dtype=torch.int32).to(device)
mfcc_features = torch.tensor(mfcc_features, dtype=torch.float32).to(device)

subset_mfcc_features = mfcc_features[sorted_indices]
subset_labels = labels[sorted_indices]

print(subset_mfcc_features.shape)


# Define the train-test split ratio for subset
train_ratio = 0.90
train_size = int(train_ratio * subset_size)
test_size = subset_size - train_size

train_dataset, test_dataset = random_split(list(zip(subset_mfcc_features, subset_labels)), [train_size, test_size])


# size = labels.shape[0]
# # Define the train-test split ratio for full length
# train_ratio = 0.95
# train_size = int(train_ratio * size)
# test_size = size - train_size


# # Create the train-test split
# train_dataset, test_dataset = random_split(list(zip(mfcc_features, labels)), [train_size, test_size])

# Separate the features and labels for train and test sets
train_mfcc_features, train_labels = zip(*train_dataset)
test_mfcc_features, test_labels = zip(*test_dataset)

# Convert to tensors
train_mfcc_features = torch.stack(train_mfcc_features).to(device)
train_labels = torch.tensor(train_labels, dtype=torch.int32).to(device)
test_mfcc_features = torch.stack(test_mfcc_features).to(device)
test_labels = torch.tensor(test_labels, dtype=torch.int32).to(device)

print(train_mfcc_features.shape, train_labels.shape)
print(test_mfcc_features.shape, test_labels.shape)


cpu
torch.Size([38546, 14, 39])


/var/folders/5z/ksmd9z810hv7hd63k2mmtgzr0000gn/T/ipykernel_4065/2480344048.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mfcc_features = torch.tensor(mfcc_features, dtype=torch.float32).to(device)


torch.Size([34691, 14, 39]) torch.Size([34691])
torch.Size([3855, 14, 39]) torch.Size([3855])


In [ ]:
import HDcompute as hd
import time
vector1 = torch.tensor(hd.make_rand_vector(seed=5, size = 10000), dtype = torch.int8).to(device)
vector2 = vector1 *-1

start = time.time()
model = hd.make_model_from_mfccs(train_mfcc_features, train_labels, vector1=vector1, vector2=vector2, 
                                  batch_size=6, device = device,
                                  seed = 42, n_gram = 2, majority_vote=True, separate_signals= True, 
                                  alpha=1, single_window=False, weighing = True)
end = time.time()
print("model training took", end-start,"seconds")

start = time.time()
preds = hd.predict(model, test_mfcc_features, majority_vote=False, weighing = True)
end = time.time()
print("prediction took", end-start,"seconds")

accuracy = torch.count_nonzero(preds == test_labels).item()*100/test_labels.shape[0]
print(f"Test Accuracy: {accuracy}%")


In [7]:
# Convert PyTorch tensors to NumPy arrays
train_mfcc_features_np = train_mfcc_features.cpu().numpy()
train_labels_np = train_labels.cpu().numpy()
test_mfcc_features_np = test_mfcc_features.cpu().numpy()
test_labels_np = test_labels.cpu().numpy()

# Reshape the features to 2D (samples, features)
train_mfcc_features_np = train_mfcc_features_np.reshape(train_mfcc_features_np.shape[0], -1)
test_mfcc_features_np = test_mfcc_features_np.reshape(test_mfcc_features_np.shape[0], -1)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import time


start = time.time()
# Train the SVM
model = SVC(kernel='rbf')
# model = LogisticRegression(max_iter=1000)
model.fit(train_mfcc_features_np, train_labels_np)
end = time.time()
print("model training took", end-start,"seconds")

start = time.time()
# Predict on the test set
test_predictions = model.predict(test_mfcc_features_np)
end = time.time()
print("prediction took", end-start,"seconds")

# Calculate accuracy
accuracy = accuracy_score(test_labels_np, test_predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [2]:
import thundersvm
from thundersvm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import time

# Assuming train and test data are prepared
# train_mfcc_features_np, train_labels_np, test_mfcc_features_np, test_labels_np

# Convert labels to integers (ThunderSVM expects numeric labels)
train_labels_np = train_labels_np.astype(int)
test_labels_np = test_labels_np.astype(int)

# Train the SVM
start = time.time()
model = SVC(kernel='rbf', gpu_id=0)  # Use GPU core 0
model.fit(train_mfcc_features_np, train_labels_np)
end = time.time()
print("Model training took", end - start, "seconds")

# Predict on the test set
start = time.time()
test_predictions = model.predict(test_mfcc_features_np)
end = time.time()
print("Prediction took", end - start, "seconds")

# Calculate accuracy
accuracy = accuracy_score(test_labels_np, test_predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


FileNotFoundError: Please build the library first!

In [ ]:
print(test_labels_np[0:10])
print(test_predictions[0:10])



In [ ]:
import torch
from collections import OrderedDict
from torch.utils.data import random_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np

accuracies = []

for i in range(2500, 38546, 2500):

    subset_size = i
    indices = torch.randperm(len(labels))[:subset_size]

    # Sort the indices
    sorted_indices = torch.sort(indices).values

    labels = torch.tensor(labels, dtype=torch.int32)
    mfcc_features = torch.tensor(mfcc_features, dtype=torch.float32)

    subset_mfcc_features = mfcc_features[sorted_indices]
    subset_labels = labels[sorted_indices]

    print(subset_mfcc_features.shape)


    # Define the train-test split ratio for subset
    train_ratio = 0.90
    train_size = int(train_ratio * subset_size)
    test_size = subset_size - train_size

    train_dataset, test_dataset = random_split(list(zip(subset_mfcc_features, subset_labels)), [train_size, test_size])


    # size = labels.shape[0]
    # # Define the train-test split ratio for full length
    # train_ratio = 0.95
    # train_size = int(train_ratio * size)
    # test_size = size - train_size


    # # Create the train-test split
    # train_dataset, test_dataset = random_split(list(zip(mfcc_features, labels)), [train_size, test_size])

    # Separate the features and labels for train and test sets
    train_mfcc_features, train_labels = zip(*train_dataset)
    test_mfcc_features, test_labels = zip(*test_dataset)

    # Convert to tensors
    train_mfcc_features = torch.stack(train_mfcc_features)
    train_labels = torch.tensor(train_labels, dtype=torch.int32)
    test_mfcc_features = torch.stack(test_mfcc_features)
    test_labels = torch.tensor(test_labels, dtype=torch.int32)

    print(train_mfcc_features.shape, train_labels.shape)
    print(test_mfcc_features.shape, test_labels.shape)


    # Convert to tensors
    # Convert PyTorch tensors to NumPy arrays
    train_mfcc_features_np = train_mfcc_features.numpy()
    train_labels_np = train_labels.numpy()
    test_mfcc_features_np = test_mfcc_features.numpy()
    test_labels_np = test_labels.numpy()



    # Reshape the features to 2D (samples, features)
    train_mfcc_features_np = train_mfcc_features_np.reshape(train_mfcc_features_np.shape[0], -1)
    test_mfcc_features_np = test_mfcc_features_np.reshape(test_mfcc_features_np.shape[0], -1)

    # Train the SVM
    model = SVC(kernel='rbf')
    # model = LogisticRegression(max_iter=1000)
    model.fit(train_mfcc_features_np, train_labels_np)
    # Predict on the test set
    test_predictions = model.predict(test_mfcc_features_np)

    # Calculate accuracy
    accuracy = accuracy_score(test_labels_np, test_predictions)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    accuracies.append(accuracy)


In [9]:
import pandas as pd
df = pd.DataFrame(accuracies, columns=["Accuracy"])
df.to_csv("SVM_accuracies.csv", index=False)

In [ ]:
import torch
from collections import OrderedDict
from torch.utils.data import random_split
import HDcompute as hd
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)


vector1 = torch.tensor(hd.make_rand_vector(seed=3, size = 10000), dtype = torch.int8).to(device)
vector2 = vector1 * -1

hd_accuracies = []


for i in range(2500, 38546, 2500):

    subset_size = i
    indices = torch.randperm(len(labels))[:subset_size]

    # Sort the indices
    sorted_indices = torch.sort(indices).values

    labels = torch.tensor(labels, dtype=torch.int32)
    mfcc_features = torch.tensor(mfcc_features, dtype=torch.float32)

    subset_mfcc_features = mfcc_features[sorted_indices]
    subset_labels = labels[sorted_indices]

    print(subset_mfcc_features.shape)


    # Define the train-test split ratio for subset
    train_ratio = 0.90
    train_size = int(train_ratio * subset_size)
    test_size = subset_size - train_size

    train_dataset, test_dataset = random_split(list(zip(subset_mfcc_features, subset_labels)), [train_size, test_size])

    # Separate the features and labels for train and test sets
    train_mfcc_features, train_labels = zip(*train_dataset)
    test_mfcc_features, test_labels = zip(*test_dataset)

    # Convert to tensors
    train_mfcc_features = torch.stack(train_mfcc_features).to(device)
    train_labels = torch.tensor(train_labels, dtype=torch.int32).to(device)
    test_mfcc_features = torch.stack(test_mfcc_features).to(device)
    test_labels = torch.tensor(test_labels, dtype=torch.int32).to(device)


    print(train_mfcc_features.shape, train_labels.shape)
    print(test_mfcc_features.shape, test_labels.shape)

    model = hd.make_model_from_mfccs(train_mfcc_features, train_labels, vector1=vector1, vector2=vector2, 
                                  batch_size=4, device = device,
                                  seed = 42, n_gram = 2, majority_vote=True, separate_signals= True, 
                                  alpha=1, single_window=False, weighing = True)

    preds = hd.predict(model, test_mfcc_features, majority_vote=False, weighing = True)

    accuracy = torch.count_nonzero(preds == test_labels).item()*100/test_labels.shape[0]

    print(f"Test Accuracy: {accuracy}%")

    hd_accuracies.append(accuracy)



In [ ]:
import numpy as np

# Example numpy arrays (replace with your actual data)
accuracies_np = np.asanyarray( hd_accuracies)  # Replace with your predictions array


# Save to CSV
np.savetxt("HD_accuracies.csv", accuracies_np, delimiter=",", header="Accuracy", comments="",  fmt="%.2f")

print("CSV file saved successfully.")

In [13]:
import torch
from collections import OrderedDict
from torch.utils.data import random_split
import HDcompute as hd
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)




hd_accuracies_size = []
vector_sizes = [2500, 5000, 7500, 10000, 15000, 20000, 25000, 30000] 



for i in vector_sizes:

    vector1 = torch.tensor(hd.make_rand_vector(seed=3, size = i), dtype = torch.int8).to(device)
    vector2 = vector1 * -1
    print("size:", i)

    subset_size = 10000
    indices = torch.randperm(len(labels))[:subset_size]


    labels = torch.tensor(labels, dtype=torch.int32)
    mfcc_features = torch.tensor(mfcc_features, dtype=torch.float32)

    subset_mfcc_features = mfcc_features[indices]
    subset_labels = labels[indices]


    # Define the train-test split ratio for subset
    train_ratio = 0.90
    train_size = int(train_ratio * subset_size)
    test_size = subset_size - train_size

    train_dataset, test_dataset = random_split(list(zip(subset_mfcc_features, subset_labels)), [train_size, test_size])

    # Separate the features and labels for train and test sets
    train_mfcc_features, train_labels = zip(*train_dataset)
    test_mfcc_features, test_labels = zip(*test_dataset)

    # Convert to tensors
    train_mfcc_features = torch.stack(train_mfcc_features).to(device)
    train_labels = torch.tensor(train_labels, dtype=torch.int32).to(device)
    test_mfcc_features = torch.stack(test_mfcc_features).to(device)
    test_labels = torch.tensor(test_labels, dtype=torch.int32).to(device)


    model = hd.make_model_from_mfccs(train_mfcc_features, train_labels, vector1=vector1, vector2=vector2, 
                                  batch_size=4, device = device,
                                  seed = 8, n_gram = 2, majority_vote=True, separate_signals= True, 
                                  alpha=1, single_window=False, weighing = True)

    preds = hd.predict(model, test_mfcc_features, majority_vote=False, weighing = True)

    accuracy = torch.count_nonzero(preds == test_labels).item()*100/test_labels.shape[0]

    print(f"Test Accuracy: {accuracy}%")

    hd_accuracies_size.append(accuracy)



mps
size: 2500


/var/folders/5z/ksmd9z810hv7hd63k2mmtgzr0000gn/T/ipykernel_85318/3334103136.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.int32)
/var/folders/5z/ksmd9z810hv7hd63k2mmtgzr0000gn/T/ipykernel_85318/3334103136.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mfcc_features = torch.tensor(mfcc_features, dtype=torch.float32)


Test Accuracy: 74.0%
size: 5000
Test Accuracy: 76.9%
size: 7500
Test Accuracy: 76.7%
size: 10000
Test Accuracy: 78.2%
size: 15000
Test Accuracy: 77.9%
size: 20000
Test Accuracy: 79.1%
size: 25000
Test Accuracy: 78.2%
size: 30000
Test Accuracy: 78.7%


In [14]:
import numpy as np

# Example numpy arrays (replace with your actual data)
accuracies_size_np = np.asanyarray( hd_accuracies_size)  # Replace with your predictions array


# Save to CSV
np.savetxt("HD_accuracies_over_size.csv", accuracies_size_np, delimiter=",", header="Accuracy", comments="",  fmt="%.2f")

print("CSV file saved successfully.")

CSV file saved successfully.


In [9]:
print(accuracies_size_np)
print(hd_accuracies_size)

NameError: name 'accuracies_size_np' is not defined